# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

# Inicialização do navegador

In [1]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

ModuleNotFoundError: No module named 'config'

In [138]:
produto = 'iphone 12 64 gb'
lista_produto = produto.split(" ")
print(lista_produto)
print(produto)

['iphone', '12', '64', 'gb']
iphone 12 64 gb


# Definição das funções de busca google shopping e do buscapé

In [139]:
import time


def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos


def verficar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    return tem_todos_termos_produtos



def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
   
    #Tratamento
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #Entra no google e fazer a busca
    nav.get("https://www.google.com/")
    nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

    #Entrar na aba shopping:
    elementos = nav.find_elements("class name", "hdtb-mitem")

    for elemento in elementos:
        if "Shopping" in elemento.text: 
            elemento.click()
            break

    #Pegar informações do produto
    lista_resultados = nav.find_elements("class name", "i0X6df")

    for resultado in lista_resultados:
        nome = resultado.find_element("class name", "tAxDx").text
        nome = nome.lower()

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
               
        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verficar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # Selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        #if tem_termos_banidos == False and tem_todos_termos_produtos == True:
        

        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = resultado.find_element("class name", "a8Pemb").text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            # Verificar se o preço está dentro do preço minimo e máximo
            if preco_minimo <= preco <= preco_maximo:
                #Lógica para pegar o link, através do parâmetro "href", não foi possível Dessa a variavel elemento_referencia recebe a ("class name", "bONr3b"), que é childre do linkE pegar o "href" atraves do parent "("xpath", "..")"
                elemento_referencia = resultado.find_element("class name", "bONr3b")
                elemento_pai = elemento_referencia.find_element("xpath", "..")
                link = elemento_pai.get_attribute('href')
                #print(preco, nome, link)
                lista_ofertas.append((nome, preco, link))
    
    
    return lista_ofertas





def busca_buscape (nav, produto, termos_banidos, preco_minimo, preco_maximo):
   
    # tratamento
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_nome_produto = produto.split(" ")
    lista_termos_banidos = termos_banidos.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    
    #Buscar no Buscape
    nav.get('https://www.buscape.com.br/')
    nav.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    
    #Pegar os resultados
    while len(nav.find_elements('class name', 'Select_Select__1S7HV')) <1:
        time.sleep(1)
    lista_resultados = nav.find_elements('class name', 'SearchCard_ProductCard_Inner__7JhKb')
    
   
    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element('class name', 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        link = resultado.get_attribute("href")
    
        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
               
        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verficar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
    
    
        #Analisar se o preço está entre o preço minímo e o preço máximo
    try:
        if not tem_termos_banidos and tem_todos_termos_produtos:
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
            preco = float(preco)
            #Se o preço ta entre preco_minimo e preco_maximo
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))
    except:
        pass
  
        
       
           
    
    #Retornar a lista de ofertas do buscape
    return lista_ofertas
            

# Construção lista de ofertas

In [140]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]


    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['Produto', 'Preço', 'Link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
        
display(tabela_ofertas)
time.sleep(10)
nav.quit()


,Produto,Preço,Link
0,"apple iphone 12, 64 gb - preto",3779.10,https://www.google.com/url?url=https://www.ama...
1,"vitrine: iphone 12 apple 64gb preto tela 6,1 c...",3295.55,https://www.google.com/url?url=https://www.car...
2,iphone 12 64gb - azul - estou zerado,3059.10,https://www.google.com/url?url=https://www.tro...
3,usado: iphone 12 64gb preto bom - trocafone,3112.73,https://www.google.com/url?url=https://www.car...
4,iphone 12 64 gb vitrine original - azul apple,3399.00,https://www.google.com/url?url=https://produto...
...,...,...,...
3,keychron k3 pro qmk/via (hot-swappable) rgb ba...,1149.00,https://www.google.com/url?url=https://keychro...
4,keychron k3 pro qmk/via wireless mechanical ke...,1056.00,https://www.google.com/url?url=https://keychro...
5,teclado keychron k3 pro rgb switch brown hotsw...,1369.00,https://www.google.com/url?url=https://produto...
6,teclado keychron k3 pro gateron lp red hotswap...,1287.00,https://www.google.com/url?url=https://produto...


# Exportando para Excel

In [141]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

# Enviando e-mail

In [142]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # Configurações do servidor de email
    servidor_smtp = 'smtp.office365.com'
    porta = 587
    seu_email = 'thiarly.cavalcante@live.com'
    sua_senha = 'xxxxxxx'

    # Criando o objeto MIMEMultipart para enviar o e-mail
    msg = MIMEMultipart()
    msg['From'] = 'thiarly.cavalcante@live.com'
    msg['To'] = 'thiarly.cavalcante@gmail.com'
    msg['Subject'] = 'Produto(s) Encontrado(s) na faixa de preço desejada'

    # Adicionando o conteúdo do e-mail
    texto_html = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    msg.attach(MIMEText(texto_html, 'html'))

    # Enviando o e-mail
    with smtplib.SMTP(servidor_smtp, porta) as server:
        server.starttls()
        server.login(seu_email, sua_senha)
        server.sendmail(seu_email, 'thiarly.cavalcante@gmail.com', msg.as_string())

nav.quit()
